In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install chromadb
!pip install pypdf
!pip install unstructured==0.7.12

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/ce/e1/59fc4dbe3a72be422c62ca96e670d590a0f7b6c795fdd0c3d757736a1a19/langchain-0.0.274-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/91/2b/92aadcea86b9ebd681de0b6b2cbfa75193227e607893cfb5feea0cefc461/SQLAlchemy-2.0.20-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached SQLAlchemy-2.0.20-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.4 kB)
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab08b6c9c1125e8bcb9d76f8b3018d101b5f877b386c/dataclasses_json-0.5.14-py3-none-any.whl.metadata
  Using cached dataclasses_json-0.5.14-py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for langsmith<0.1.0,>=0.0.21 from https://files.pythonhosted.org/packages/94/af/c01937934a8d6

In [6]:
import os

os.environ['OPENAI_API_KEY'] = 'sk------'

In [7]:
path_for_directory_of_pdf = '/home/jovyan/workspace/MangrovePDFs'

In [9]:
from langchain.vectorstores import Chroma
from langchain.vectorstores.vectara import VectaraRetriever
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader

#pdf_loader = DirectoryLoader(path_for_directory_of_pdf, glob="**/*.pdf")
#loaders = [pdf_loader]
#documents = []

pdf_loader = DirectoryLoader(path_for_directory_of_pdf, glob="**/*.pdf")
loaders =[pdf_loader]
documents = []

#for loader in loaders:
    #documents.extend(loader.load())
    
for loader in loaders:
    documents.extend(loader.load())
    

from langchain.text_splitter import CharacterTextSplitter

#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#documents = text_splitter.split_documents(documents)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

Created a chunk of size 1653, which is longer than the specified 1000
Created a chunk of size 2803, which is longer than the specified 1000
Created a chunk of size 1173, which is longer than the specified 1000
Created a chunk of size 1558, which is longer than the specified 1000
Created a chunk of size 1192, which is longer than the specified 1000
Created a chunk of size 2023, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1344, which is longer than the specified 1000
Created a chunk of size 1506, which is longer than the specified 1000
Created a chunk of size 1817, which is longer than the specified 1000
Created a chunk of size 1798, which is longer than the specified 1000
Created a chunk of size 1444, which is longer than the specified 1000
Created a chunk of size 1138, which is longer than the specified 1000
Created a chunk of size 1765, which is longer than the specified 1000
Created a chunk of s

In [10]:
import os

if not os.path.exists(f'{path_for_directory_of_pdf}db'):
    os.mkdir(f'{path_for_directory_of_pdf}db')

#embeddings = OpenAIEmbeddings()
#vectorstore = Chroma.from_documents(
    #documents, embeddings, persist_directory=f"{path_for_directory_of_pdf}db")
#vectorstore.persist()
#vectorstore = None

embeddings =OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents, embeddings, persist_directory=f"{path_for_directory_of_pdf}db")

vectorstore.persist()
vectorstore = None

In [11]:
vectorstore = Chroma(
    persist_directory=f'{path_for_directory_of_pdf}db', embedding_function=embeddings)

In [12]:
llm = OpenAI(model_name="gpt-3.5-turbo-16k")

memory = ConversationBufferMemory(llm=llm, memory_key="chat_history")
qa = ConversationalRetrievalChain.from_llm(
    llm, vectorstore.as_retriever(), memory=memory, get_chat_history=lambda h: h)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [13]:
question = "What are mangroves?"
result = qa({"question": question})
print(result)

{'question': 'What are mangroves?', 'chat_history': '', 'answer': 'Mangroves are forested wetlands that are uniquely adapted to the intertidal zone, found in the coastal zones of more than 118 countries in the tropics, subtropics, and temperate regions. They provide natural resources to local populations, including food and timber. However, their extent has been reduced through various processes, and many have been degraded or fragmented, particularly in Southeast Asia. Mangroves are increasingly affected by climatic fluctuations, and they are recognized for their role in food provision, coastal protection, biodiversity reserves, and carbon storage. Protection and restoration efforts are being made to ensure their conservation and sustainable management.'}


In [14]:
question = "What are mangroves?"
result = qa({"question": question})
print("question", question)
print("answer",result['answer'])

question What are mangroves?
answer Mangroves play a role in food provision, coastal protection, biodiversity reserves, and carbon storage.
